# **Data Scrapping From Hello Work**

In [ ]:
pip install pandas openpyxl

# **Data Collection**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# ==== HEADERS ====
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# ==== FONCTION : RÉCUPÉRER LES LIENS DES OFFRES ====
def get_offer_links_from_pages(base_url, max_pages=3):
    all_links = set()

    for page in range(1, max_pages + 1):
        page_url = f"{base_url}&p={page}"
        print(f"Page {page} → {page_url}")

        try:
            response = requests.get(page_url, headers=headers)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Erreur page {page} : {e}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")

        for link in soup.find_all("a", href=True):
            href = link["href"]
            if "/fr-fr/emplois/" in href:
                full_link = "https://www.hellowork.com" + href
                all_links.add(full_link)

        time.sleep(1)

    return list(all_links)


# ==== FONCTION : SCRAPER UNE OFFRE ====
def scrape_offer_details(url):
    print(f"Scraping : {url}")
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur : {e}")
        return {}

    soup = BeautifulSoup(response.content, "html.parser")

    # === Infos principales
    job_title_el = soup.find("span", {"data-cy": "jobTitle"})
    job_title = job_title_el.get_text(strip=True) if job_title_el else ""

    entreprise = lieu = contrat = ""

    if job_title_el:
        h1_block = job_title_el.find_parent("h1")
        h1_spans = h1_block.find_all("span")
        if len(h1_spans) >= 2:
            entreprise = h1_spans[1].get_text(strip=True)

        next_spans = h1_block.find_all_next("span", class_="tw-inline-flex tw-typo-m tw-text-grey", limit=2)
        if len(next_spans) > 0:
            lieu = next_spans[0].get_text(strip=True)
        if len(next_spans) > 1:
            contrat = next_spans[1].get_text(strip=True)

    # === Description du poste
    try:
        description_tag = soup.find('h2', string=lambda s: s and 'missions' in s.lower())
        description = ""
        if description_tag:
            description_elements = []
            for elem in description_tag.find_all_next():
                if elem.name == "h2":
                    break
                if elem.name in ["p", "ul", "ol"]:
                    description_elements.append(elem.get_text(" ", strip=True))
            description = "\n".join(description_elements)
    except Exception as e:
        print(f"Erreur description : {e}")
        description = ""

    # === Profil recherché
    try:
        profil_tag = soup.find('h2', string=lambda s: s and 'profil' in s.lower())
        profil = ""
        if profil_tag:
            profil_elements = []
            for elem in profil_tag.find_all_next():
                if elem.name == "h2":
                    break
                if elem.name in ["p", "ul", "ol"]:
                    profil_elements.append(elem.get_text(" ", strip=True))
            profil = "\n".join(profil_elements)
    except Exception as e:
        print(f"Erreur profil : {e}")
        profil = ""

    # === Infos secondaires
    infos = soup.select_one("section.tw-mb-8 ul")
    infos_list = infos.find_all("li") if infos else []

    secteur = niveau = experience = ""
    teletravail = False

    for li in infos_list:
        txt = li.get_text(strip=True).lower()
        if "secteur" in txt or "service" in txt:
            secteur = li.get_text(strip=True)
        elif "exp." in txt:
            experience = li.get_text(strip=True)
        elif "bac" in txt:
            niveau = li.get_text(strip=True)
        elif "télétravail" in txt:
            teletravail = True

    salaire_el = soup.select_one(".tw-tag-attractive-s")
    salaire = salaire_el.get_text(strip=True) if salaire_el else ""

    return {
        "URL": url,
        "Titre": job_title,
        "Entreprise": entreprise,
        "Lieu": lieu,
        "Contrat": contrat,
        "Salaire": salaire,
        "Secteur": secteur,
        "Télétravail": teletravail,
        "Niveau": niveau,
        "Expérience": experience,
        "Description": description,
        "Profil": profil
    }


# ==== MAIN ====
if __name__ == "__main__":
    search_url = "https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france"
    offer_links = get_offer_links_from_pages(search_url, max_pages=20)

    print(f"Total liens récupérés : {len(offer_links)}")

    data = []

    try:
        for i, link in enumerate(offer_links):
            result = scrape_offer_details(link)
            if result:
                data.append(result)
            time.sleep(1)
    except KeyboardInterrupt:
        print("Interruption manuelle. Enregistrement des résultats partiels...")

    if data:
        df = pd.DataFrame(data)
        df.to_csv("hellowork_offres.csv", index=False)
        print("Fichier généré : hellowork_offres.csv")
    else:
        print("Aucune donnée trouvée.")

Page 1 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=1
Page 2 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=2
Page 3 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=3
Page 4 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=4
Page 5 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=5
Page 6 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=6
Page 7 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=7
Page 8 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=8
Page 9 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=france&p=9
Page 10 → https://www.hellowork.com/fr-fr/emploi/recherche.html?k=ingenieur+informatique&l=

Converting Description & Profil to Lowercase in order to extract skills from

In [ ]:
import pandas as pd


df = pd.read_csv("/content/hellowork_offres.csv")

# Mettre en minuscules uniquement les colonnes "Description" et "Profil"
for col in ["Description", "Profil"]:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)

)
df.to_csv("hellowork_offres.csv", index=False)

print("Les colonnes 'Description' et 'Profil' ont été mises en minuscules.")

Les colonnes 'Description' et 'Profil' ont été mises en minuscules.


In [ ]:
!pip install -U spacy
!python -m spacy download fr_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 10.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Extracting skills from Description and Profil based on a skills file

In [ ]:
import pandas as pd
import spacy
import re


nlp = spacy.load("fr_core_news_md")

#  Charger la liste des compétences depuis ton fichier CSV
skills_df = pd.read_csv("/content/data_base_competences.csv")
SKILLS = set(skill.strip().lower() for skill in skills_df['hardskills'].dropna())

#  Fonction de détection simple
def detect_skills(text):
    if not isinstance(text, str):
        return "Non spécifié"

    text = text.lower()
    found = set()

    for skill in SKILLS:
        # Si la compétence contient des caractères spéciaux, ne pas utiliser \b
        if any(char in skill for char in ['+', '#', '.', '_']):
            pattern = re.escape(skill)
        else:
            pattern = r'\b' + re.escape(skill) + r'\b'

        if re.search(pattern, text):
            found.add(skill)

    return ", ".join(sorted(found)) if found else "Non spécifié"


df = pd.read_csv("/content/hellowork_offres.csv")


df["Compétences"] = df.apply(
    lambda row: detect_skills(f"{row['Description']} {row['Profil']}"), axis=1
)

df.to_csv("/content/hellowork_offres_competences.csv", index=False)

print(" Extraction terminée. Fichier enregistré : hellowork_offres_competences.csv")


 Extraction terminée. Fichier enregistré : hellowork_offres_competences.csv


# **Data Pre-Processing**

Removing records with no skills detected and non used columnn :Description &Profil



In [ ]:
import pandas as pd

In [ ]:

df = pd.read_csv("/content/hellowork_offres_competences.csv")


df = df[df["Compétences"] != "Non spécifié"]

df = df.drop(columns=["Description","Profil"])


df.to_csv("new_data_offers.csv", index=False)

Nb competences in each offer

In [ ]:
df = pd.read_csv("/content/new_data_offers.csv")


df["Nb_Competences"] = df["Compétences"].fillna("").apply(lambda x: len(x.split()))


df.to_csv("new_data_offers.csv", index=False)

Converting all the salaries to /year

In [ ]:
import pandas as pd
import numpy as np
import re

def convertir_salaire(s):
    try:
        if pd.isna(s):
            return None


        s = s.lower().replace('\u202f', ' ').replace('\xa0', ' ').strip()
        s = s.replace("/an", "/ an").replace("/mois", "/ mois").replace("/jour", "/ jour").replace("/h", "/ h").replace("/heure", "/ h")

        # Corriger les formats comme "12-15 / heure" ou "13 - 14 / heure"
        if "/ h" in s and "€" not in s:
            s = s.replace("/ h", "€ / h")
        if "/ jour" in s and "€" not in s:
            s = s.replace("/ jour", "€ / jour")
        if "/ mois" in s and "€" not in s:
            s = s.replace("/ mois", "€ / mois")
        if "/ an" in s and "€" not in s:
            s = s.replace("/ an", "€ / an")


        s = re.sub(r"(\d)\s+(\d{3})", r"\1\2", s)

        # Cas / an
        if "/ an" in s:
            parts = s.replace("€ / an", "").strip().split("-")
            if len(parts) == 2:
                min_val = float(parts[0].strip().replace(" ", "").replace(",", "."))
                max_val = float(parts[1].strip().replace(" ", "").replace(",", "."))
            else:
                val = float(parts[0].strip().replace(" ", "").replace(",", "."))
                min_val = max_val = val
            return f"{min_val:,.2f} - {max_val:,.2f} € / an".replace(",", " ").replace(".", ",")

        # Cas / mois → x12
        elif "/ mois" in s:
            parts = s.replace("€ / mois", "").strip().split("-")
            if len(parts) == 2:
                min_val, max_val = [float(p.strip().replace(" ", "").replace(",", ".")) for p in parts]
            else:
                val = float(parts[0].strip().replace(" ", "").replace(",", "."))
                min_val = max_val = val
            return f"{min_val*12:,.2f} - {max_val*12:,.2f} € / an".replace(",", " ").replace(".", ",")

        # Cas / h ou / heure → x35*52
        elif re.search(r"/ ?(h|heure)", s):
            s = s.replace("€ / h", "").replace("€ / heure", "").replace("/ h", "").replace("/ heure", "").strip()
            parts = s.split("-")
            if len(parts) == 2:
                min_val, max_val = [float(p.strip().replace(" ", "").replace(",", ".")) for p in parts]
            else:
                val = float(parts[0].strip().replace(" ", "").replace(",", "."))
                min_val = max_val = val
            heures_annuelles = 35 * 52
            return f"{min_val*heures_annuelles:,.2f} - {max_val*heures_annuelles:,.2f} € / an".replace(",", " ").replace(".", ",")

        # Cas / jour → x220
        elif "/ jour" in s:
            parts = s.replace("€ / jour", "").strip().split("-")
            if len(parts) == 2:
                min_val, max_val = [float(p.strip().replace(" ", "").replace(",", ".")) for p in parts]
            else:
                val = float(parts[0].strip().replace(" ", "").replace(",", "."))
                min_val = max_val = val
            jours_annuels = 220
            return f"{min_val*jours_annuels:,.2f} - {max_val*jours_annuels:,.2f} € / an".replace(",", " ").replace(".", ",")

        return s
    except:
        return s


df = pd.read_csv("/content/new_data_offers.csv")


df["Salaire"] = df["Salaire"].apply(convertir_salaire)

df.to_csv("/content/new_data_offers.csv", index=False)


Ensure all salaries are annual

In [ ]:
df = pd.read_csv("/content/new_data_offers.csv")

# Filtrer les lignes où "Salaire" ne contient pas "an"
salaire_non_annuel = df[df["Salaire"].notna() & ~df["Salaire"].str.contains("an", case=False, na=False)]

print("Nombre de salaires non annuels :", salaire_non_annuel.shape[0])
display(salaire_non_annuel[["Salaire"]])

Nombre de salaires non annuels : 0


,Salaire


# Missing Values treatment

In [63]:
import pandas as pd
df = pd.read_csv("/content/new_data_offers.csv")

# Compter les valeurs manquantes par colonne
missing_exp = df["Expérience"].isna().sum()
missing_secteur = df["Secteur"].isna().sum()
missing_niveau = df["Niveau"].isna().sum()
missing_teletravail = df["Télétravail"].isna().sum()
missing_salaire = df["Salaire"].isna().sum()
missing_entreprise = df["Entreprise"].isna().sum()
missing_lieu = df["Lieu"].isna().sum()
missing_contrat = df["Contrat"].isna().sum()

# Affichage lisible et regroupé
print(" Valeurs manquantes par colonne :\n")
print(f"• Expérience     : {missing_exp}")
print(f"• Secteur        : {missing_secteur}")
print(f"• Niveau         : {missing_niveau}")
print(f"• Télétravail    : {missing_teletravail}")
print(f"• Salaire        : {missing_salaire}")
print(f"• Entreprise     : {missing_entreprise}")
print(f"• Lieu           : {missing_lieu}")
print(f"• Contrat        : {missing_contrat}")

 Valeurs manquantes par colonne :

• Expérience     : 145
• Secteur        : 139
• Niveau         : 2
• Télétravail    : 0
• Salaire        : 66
• Entreprise     : 3
• Lieu           : 0
• Contrat        : 0


Replace each Experience based on same type of offer

In [64]:
import matplotlib.pyplot as plt


df = pd.read_csv("/content/new_data_offers.csv")

# Remplir 'Expérience' si manquante dependamment du 'Titre'
df.loc[df["Expérience"].isna() & df["Titre"].str.contains("ingénieur|Ingenieur|Consultant|Engineer|spécialiste", case=False, na=False), "Expérience"] = "Exp. 1 à 7 ans"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains("Stage|stagiaire", case=False, na=False), "Expérience"] = "0"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains( r"alternance|alternant|apprentissage|apprenti", case=False, na=False), "Expérience"] = "0"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains( r"Informaticien", case=False, na=False), "Expérience"] = "Exp. 1 à 5 ans"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains( r"Architecte", case=False, na=False), "Expérience"] = "Exp. 8 à 12 ans"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains( r"Senior", case=False, na=False), "Expérience"] = "Exp. 5 à 8 ans"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains( r"Expert", case=False, na=False), "Expérience"] = "Exp. 4 à 10 ans"
df.loc[df["Expérience"].isna() & df["Titre"].str.contains( r"Doctorant", case=False, na=False), "Expérience"] = "Exp. 0 à 2 ans"

df.to_csv("/content/new_data_offers.csv", index=False)


In [67]:
missing_exp_df = df[df["Expérience"].isna()]

print("Nombre de lignes avec 'Expérience' manquante :", missing_exp_df.shape[0])

from IPython.display import display
display(missing_exp_df)

Nombre de lignes avec 'Expérience' manquante : 5


,URL,Titre,Entreprise,Lieu,Contrat,Salaire,Secteur,Télétravail,Niveau,Expérience,Compétences,Nb_Competences
56,https://www.hellowork.com/fr-fr/emplois/616083...,Pentester Web & Infra H/F,BPCE Infogérance & Technologies,Paris - 75,CDI,"55 000,00 - 65 000,00 € / an",NaN,True,Bac +5,NaN,"agile, framework, javascript, python, validation",5
61,https://www.hellowork.com/fr-fr/emplois/640739...,Chargé de Gouvernance de Données H/F,Thélem assurances,Chécy - 45,CDD,"21 600,00 - 21 600,00 € / an",NaN,False,Bac +5,NaN,"automatisation, excel, gestion des incidents, ...",9
143,https://www.hellowork.com/fr-fr/emplois/640370...,Administrateur Sécurité Informatique H/F,LABASTIE HEADHUNTING,Paris - 75,CDI,"55 000,00 - 65 000,00 € / an",Services aux Entreprises,True,Bac +5,NaN,"hardening, office 365, réseaux",4
273,https://www.hellowork.com/fr-fr/emplois/642707...,Hardware - Software Cybersecurity Of Embedded ...,CEA,Saclay - 91,CDD,"35 000,00 - 50 000,00 € / an",NaN,False,Bac +5,NaN,"c, c++",2
338,https://www.hellowork.com/fr-fr/emplois/640594...,Pentesteur H/F,EMITECH GROUPE,Montigny-le-Bretonneux - 78,CDI,"55 000,00 - 65 000,00 € / an",NaN,False,Bac +5,NaN,"automatisation, cybersécurité, documentation t...",6


In [69]:

df = df.dropna(subset=["Entreprise"])


df.to_csv("/content/new_data_offers.csv", index=False)

print(f"Nombre de lignes restantes : {len(df)}")

Nombre de lignes restantes : 452


In [70]:
missing_entreprise = df["Entreprise"].isna().sum()
print(f"• Entreprise     : {missing_entreprise}")

• Entreprise     : 0


Replacing Salaries by mean

In [75]:
import pandas as pd
import numpy as np


df = pd.read_csv("/content/new_data_offers.csv")


def extraire_moyenne_salaire(s):
    try:
        if pd.isna(s):
            return np.nan
        s = s.replace("€", "").replace("/ an", "").replace(",", ".").replace('\u202f', ' ').replace('\xa0', ' ').strip()
        parts = s.split("-")
        if len(parts) == 2:
            min_val = float(parts[0].strip().replace(" ", ""))
            max_val = float(parts[1].strip().replace(" ", ""))
            return (min_val + max_val) / 2
        else:
            val = float(parts[0].strip().replace(" ", ""))
            return val
    except:
        return np.nan


df["Salaire_moyen_temp"] = df["Salaire"].apply(extraire_moyenne_salaire)


moyenne_globale = round(df["Salaire_moyen_temp"].mean(), 2)


texte_moyenne = f"{moyenne_globale:,.2f} - {moyenne_globale:,.2f} € / an".replace(",", " ").replace(".", ",")


df["Salaire"] = df["Salaire"].fillna(texte_moyenne)


df = df.drop(columns=["Salaire_moyen_temp"])


df.to_csv("/content/new_data_offers.csv", index=False)

print(f" Moyenne insérée dans les valeurs manquantes : {texte_moyenne}")


 Moyenne insérée dans les valeurs manquantes : 41 160,07 - 41 160,07 € / an


In [76]:
missing_entreprise = df["Salaire"].isna().sum()
print(f"• Salaire    : {missing_entreprise}")

• Salaire    : 0


# Organize Types of variables

Converting salary to numerical

In [ ]:
import numpy as np

In [54]:

df = pd.read_csv("/content/new_data_offers.csv")


def extraire_salaire_moyen(s):
    try:
        if pd.isna(s):
            return np.nan
        s = s.replace('\u202f', ' ').replace('\xa0', ' ').strip().lower()
        s = s.replace("€", "").replace("/ an", "").strip()
        parts = s.split("-")
        if len(parts) == 2:
            min_val = float(parts[0].strip().replace(" ", "").replace(",", "."))
            max_val = float(parts[1].strip().replace(" ", "").replace(",", "."))
            return round((min_val + max_val) / 2, 2)
        else:
            val = float(parts[0].strip().replace(" ", "").replace(",", "."))
            return round(val, 2)
    except:
        return np.nan


df["Salaire"] = df["Salaire"].apply(extraire_salaire_moyen)
r
df.to_csv("/content/new_data_offers.csv", index=False)

df[["Salaire"]].head()

,Salaire
0,13729.74
1,NaN
2,50000.00
3,52500.00
4,36000.00


Separate City & Depart

In [77]:

df[["Ville", "Département"]] = df["Lieu"].str.extract(r"^(.*?)\s*-\s*(\d+)$")


df["Département"] = pd.to_numeric(df["Département"], errors="coerce")


df = df.drop(columns=["Lieu"])


colonnes_ordre = [
    "URL", "Titre", "Entreprise", "Ville", "Département", "Contrat", "Salaire", "Secteur",
    "Télétravail", "Niveau", "Expérience", "Compétences", "Nb_Competences"
]
df = df[colonnes_ordre]


df.to_csv("/content/new_data_offers.csv", index=False)


df.head()

,URL,Titre,Entreprise,Ville,Département,Contrat,Salaire,Secteur,Télétravail,Niveau,Expérience,Compétences,Nb_Competences
0,https://www.hellowork.com/fr-fr/emplois/641285...,Alternance Apprenti Informatique Industrielle H/F,Tereos,Marckolsheim,67.0,Alternance,"5 837,88 - 21 621,60 € / an",NaN,False,Bac +5,0,"c, excel, informatique industrielle, réseau, r...",7
1,https://www.hellowork.com/fr-fr/emplois/635705...,Ingénieur de Production Informatique H/F,Open,Tours,37.0,CDI,"41 160,07 - 41 160,07 € / an",Secteur informatique • ESN,True,Bac +5,Exp. 1 à 7 ans,"analyse technique, automatisation, cloud, devo...",11
2,https://www.hellowork.com/fr-fr/emplois/638487...,Ingénieur Secops - Cybersécurité H/F,Lutessa,La Défense,92.0,CDI,"50 000,00 - 55 000,00 € / an",Secteur informatique • ESN,True,Bac +5,Exp. 1 à 7 ans,"analyse technique, cybersécurité",3
3,https://www.hellowork.com/fr-fr/emplois/576469...,Ingénieur Dev - DevOps H/F,Meritis,Biot,6.0,CDI,"30 000,00 - 42 000,00 € / an",Secteur informatique • ESN,False,Bac +5,Exp. 1 à 7 ans,"agile, ansible, automatisation, docker, gestio...",12
4,https://www.hellowork.com/fr-fr/emplois/635987...,Expert Informatique Industrielle H/F,Coris Innovation,Grenoble,38.0,CDI,"45 000,00 - 50 000,00 € / an",Services aux Entreprises,False,Bac +5,Exp. 4 à 10 ans,"api, automatisation, erp, ia, ihm, informatiqu...",23


Testing the types of contract

In [79]:
df = pd.read_csv("/content/new_data_offers.csv")


contrats_valides = ["intérim", "cdi", "cdd", "indépendant", "alternance", "stage"]


contrats_autres = df[df["Contrat"].notna() & ~df["Contrat"].str.lower().isin(contrats_valides)]


print(f"Nombre de contrats hors CDI, CDD, Intérim, Indépendant, Alternance, Stage : {contrats_autres.shape[0]}")
display(contrats_autres[["Contrat"]])



Nombre de contrats hors CDI, CDD, Intérim, Indépendant, Alternance, Stage : 0


,Contrat


Converting Niveau & Experience to Numerical

In [80]:
import pandas as pd
import re
import numpy as np


df = pd.read_csv("/content/new_data_offers.csv")


def convertir_niveau(val):
    try:
        if pd.isna(val):
            return np.nan
        # Extraire le nombre après "Bac +"
        match = re.search(r"\+?(\d+)", str(val))
        if match:
            return int(match.group(1))
        else:
            return np.nan
    except:
        return np.nan


def convertir_experience(val):
    try:
        if pd.isna(val):
            return np.nan
        # Extraire le premier chiffre
        match = re.search(r"(\d+)", str(val))
        if match:
            return int(match.group(1))
        else:
            return np.nan
    except:
        return np.nan

df["Niveau"] = df["Niveau"].apply(convertir_niveau)
df["Expérience"] = df["Expérience"].apply(convertir_experience)

df.to_csv("/content/new_data_offers.csv", index=False)

df[["Niveau", "Expérience"]].head()


,Niveau,Expérience
0,5.0,0.0
1,5.0,1.0
2,5.0,1.0
3,5.0,1.0
4,5.0,4.0


Converting salaries to Numerical

In [81]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/new_data_offers.csv")

def extraire_moyenne_salaire(s):
    try:
        if pd.isna(s):
            return np.nan
        # Nettoyage
        s = s.replace('\u202f', ' ').replace('\xa0', ' ').lower()
        s = s.replace("€", "").replace("/ an", "").strip()
        parts = s.split("-")
        if len(parts) == 2:
            min_val = float(parts[0].strip().replace(" ", "").replace(",", "."))
            max_val = float(parts[1].strip().replace(" ", "").replace(",", "."))
            return round((min_val + max_val) / 2, 2)
        else:
            val = float(parts[0].strip().replace(" ", "").replace(",", "."))
            return round(val, 2)
    except:
        return np.nan


df["Salaire"] = df["Salaire"].apply(extraire_moyenne_salaire)

df.to_csv("/content/new_data_offers.csv", index=False)


df[["Salaire"]].head()


,Salaire
0,13729.74
1,41160.07
2,52500.00
3,36000.00
4,47500.00


Converting Dept, Nv, Exp to numerical

In [82]:
df = pd.read_csv("/content/new_data_offers.csv")


df["Département"] = pd.to_numeric(df["Département"], errors="coerce").round().astype("Int64")
df["Niveau"] = pd.to_numeric(df["Niveau"], errors="coerce").round().astype("Int64")
df["Expérience"] = pd.to_numeric(df["Expérience"], errors="coerce").round().astype("Int64")


df.to_csv("/content/new_data_offers.csv", index=False)


df[["Département", "Niveau", "Expérience"]].head()

,Département,Niveau,Expérience
0,67,5,0
1,37,5,1
2,92,5,1
3,6,5,1
4,38,5,4
